<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/single_material-reproducing_CT_scan-JSON_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2025 United Kingdom Research and Innovation
#  Copyright 2025 Bangor University
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC) & Iwan Mitchell (Bangor University)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Single-material example: reproducing the CT scan of a teapot using a JSON file

They say French people drink coffee not tea.
 However, we found a teapot in the lab and scanned it. 
 It's made of a dense material (cast iron), making it interesting to scan. 
 It also has a fine wavy texture. 
 We acquired the data presented below with the [Dual Tube High Energy (DTHE) device](https://www.rx-solutions.com/en/blog/126/dthe-technology) by [RX Solutions](https://www.rx-solutions.com/) installed at the [MATEIS Laboratory](https://mateis.insa-lyon.fr/en) of [INSA-Lyon (France)](http://www.insa-lyon.fr/). The DTHE is a double tomograph designed around one rotation axis and two 300 kV X-ray beamlines. 
We'll aim to reproduce one of the scans we made. 
We will take into account the source and detector properties as finely as possible. 
We'll use the metadata generated by the device to create a JSON file that describes our virtual experiment.
It'll makes it so much easier than typing all the Python code:
Describe the simulation in a human-friendly format, load the file and there you go!
We will eventually use CIL to reconstruct the CT data with the traditional FDK algorithm.

<!-- ![Sample on the turntable of the device.](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/data/Al-part/IMG_3508-smaller.jpg?raw=1) -->

![Sample on the turntable of the device.](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/data/theiere-Fe-10mm/IMG_3458-smaller.jpg?raw=1)


<!-- ![Rendering of the simulation](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/output_data/single_material-reproducing_CT_scan/k3d_screenshot-cropped.png?raw=1) -->


<!-- Up to now, we generated 2D radiographs. 
This time we are going to simulate the computed tomography (CT) acquisition of a section of electric cable. 
A cone-beam geometry is used. 
To keep it simple, we consider an ideal case: no scintillation, no point-spread function (PSF), and a monochromatic spectrum. -->

<!-- We will use CIL to reconstruct the CT data with the traditional FDK algorithm. -->



<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the required packages.
</div>

## Aim and objectives of this session

The aim of this session is to replicate an actual scan.

To achieve this, the main objectives are to:

1. Identify the parameters are of importance in the metadata file(s) produced by the device.
2. Describe the simulation and reconstruction in a human-friendly file.
3. Run the simulation using this file instead of writing a lot of Python code.
4. Perform the reconstruction using CIL.

![Screenshot of the 3D environment using K3D](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/output_data/single_material-reproducing_CT_scan-JSON_file/k3d_screenshot-cropped.png?raw=1)

## Import packages

<!-- - `pandas`
- `xml.etree` -->

<!-- - `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `viewscad` to use OpenSCAD and create STL files
- `base64` to use save a screenshot of the K3D visualisation
- `tifffile.imread` to read the X-ray projection of the real experiment
- `xml.etree.ElementTree` to read the scanning parameters of the real experiment
- `gvxrPython3.gvxr` to simulate X-ray images
- `gvxrPython3.visualise` to use K3D to visualise the 3D environment
- `gvxrPython3.gvxr2json` to save the state of the simulation in JSON file
- `gvxrPython3.JSON2gVXRDataReader` to read the simulated data and prepare it for CIL
- `cil.recon.FDK` to reconstruct the CT volume using the FDK algorithm (3D-CBCT only) 
- `cil.recon.FBP` to reconstruct the CT volume using the FBP algorithm (2D/3D parallel geometry only) 
- `cil.io.TIFFWriter` to save the reconstructed slices as a TIFF stack
- `cil.processors.TransmissionAbsorptionConverter` to normalise the data and apply the minus log
- `cil.utilities.display.show_geometry` to display the acquisition geometry
- `cil.utilities.display.show2D` to show static 2D images
- `cil.utilities.jupyter.islicer` to show and navigate a stack of 2D images using a slider
- `cil.utilities.jupyter.link_islicer` to compare and navigate stacks of 2D images using a slider -->

In [ ]:
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?

# import math

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
import matplotlib.gridspec as gridspec

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# # Uncomment the line below to use LaTeX fonts
# # matplotlib.rc('text', usetex=True)

# import viewscad # Use OpenSCAD to create STL files

import base64

from tifffile import imread, imwrite

# import xml.etree.ElementTree as ET

from gvxrPython3 import gvxr
from gvxrPython3 import json2gvxr

# import pandas
from xml.etree import ElementTree
from gvxrPython3.utils import visualise

# Use temporary bug fix
if os.path.exists("gvxr2json.py"):
    print("Use temporary bug fix")
    import gvxr2json
#Use the file provided by gVXR's package
else:
    print("Use the file provided by gVXR's package")
    from gvxrPython3 import gvxr2json

# Use temporary bug fix
if os.path.exists("json2gvxr.py"):
    print("Use temporary bug fix")
    import json2gvxr
#Use the file provided by gVXR's package
else:
    print("Use the file provided by gVXR's package")
    from gvxrPython3 import json2gvxr


from tqdm import tqdm


from gvxrPython3.JSON2gVXRDataReader import *

from cil.recon import FDK # For CBCT
# from cil.recon import FBP # For parallel beam geometry

from cil.io import TIFFWriter

from cil.processors import TransmissionAbsorptionConverter
from cil.utilities.display import show_geometry, show2D
from cil.utilities.jupyter import islicer, link_islicer

import k3d

# from ipywidgets import interact

## Getting the data ready

Where to save the data.

In [ ]:
output_path = "output_data/single_material-reproducing_CT_scan-JSON_file"
if not os.path.exists(output_path):
    os.makedirs(output_path);

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

dataset_path = "../data/theiere-Fe-10mm"
projection_fname = "Proj/img_00598.tif"
JSON_fname = os.path.join(dataset_path, "parameters.json")

if IN_COLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

    !conda install -y -c conda-forge -c https://software.repos.intel.com/python/conda -c ccpi cil=24.2.0 ipp=2021.12 astra-toolbox=*=cuda* tigre k3d

    !pip install gvxr

    import os, urllib.request

    if not os.path.exists(os.path.join(dataset_path, "Proj")):
        os.makedirs(os.path.join(dataset_path, "Proj"));

    files = [os.path.join(dataset_path, "parameters.json"),
             os.path.join(dataset_path, "geom.csv"),
             os.path.join(dataset_path, "restore.macro"),
             os.path.join(dataset_path, "unireconstruction.xml"),
             os.path.join(dataset_path, projection_fname),
             "notebooks/gvxr2json.py",
             "notebooks/json2gvxr.py",
            ];

    for fname in files:
        url = "https://github.com/TomographicImaging/gVXR-Tutorials/raw/refs/heads/main/" + fname;
        print("Download", url);
        urllib.request.urlretrieve(url, fname);

# 1. Identify the parameters are of importance in the metadata file(s) produced by the device

## Summary of the ~~simulation~~ experimental parameters

| Parameters | Values | Units |
|------------|--------|-------|
| Source type (beam shape) | conebeam | |
| Beamline   | ?? | |
| Number of projections | ?? | |
| Tube voltage | ?? | kV |  |
| Tube current | ?? | &mu;A |  |
| Source to detector distance | ?? | mm |
| Source to object distance | ?? | mm |
| Object to detector distance | ?? | mm |
| Filter material | ?? | |
| Filter thickness | ?? | mm |
| Exposure time | ?? | sec |
| Exposure | ??| mAs |
| Pixel pitch | ?? x ?? | &mu;m |
| Image size | ?? x ?? | pixels |


<!-- | Detector orientation | [0, 0, 1] |  |
| Detector resolution | ?? &times; ?? | pixels |
| Pixel pitch | 150 &times; 150 | um | -->

The table above should contain all the parameters of interest that we need to fully describe our simulation and CT acquisition. 
It contains a lot of `??`. It's your job to complete it! For that purpose we will extract all the useful data from the metadata file. There are two of interest in the case of the DTHE:

- [unireconstruction.xml](../data/theiere-Fe-10mm/unireconstruction.xml)
- [restore.macro](../data/theiere-Fe-10mm/restore.macro)

---
## Tasks:

1. Double-click on the table above to edit it.
2. Open [unireconstruction.xml](../data/theiere-Fe-10mm/unireconstruction.xml).
3. As there are two beamlines, we need to know which one was use. 
    - Look for `Imager_1` or `Imager_2` in the `profile acquisition`.
    - As we are at it, identify how many images were captured.
    - Report the corresponding information in the table.
4. We will now record the details about the source. With the voltage and current, we'll be able to generate the spectrum and the flux.
    - Look for `voltage` or `current`.
    - Report the corresponding information in the table.
5. Do the same as above for source to detector distance (`sdd`) and the source to object distance (`sod`). Note that the distances are given in millimetres. You can see in the picture above that the distance from the source to the teapot is roughly 40 cm. 
6. Uncomment all the lines of the cell below and write down the corresponding values.
---

In [ ]:
# nb_images = 
# voltage_in_kV = 
# current_in_uA = 
# sdd_in_mm = 
# sod_in_mm = 

In [ ]:
# %load 'snippets/single_material-reproducing_CT_scan-JSON_file-01.py'

Assuming that we will centre the teapot in the middle of our world, we will need to know the object to detector distance (odd) to position the detector.


---
## Tasks:

1. Uncomment the line in the cell below,
2. Compute the `odd` using the `sdd` and `sod`.
2. Report the value in the table.
---

In [ ]:
# odd_in_mm = 

In [ ]:
# %load 'snippets/single_material-reproducing_CT_scan-JSON_file-02.py'

We are done with [unireconstruction.xml](../data/theiere-Fe-10mm/unireconstruction.xml). 
We'll have a look at [restore.macro](../data/theiere-Fe-10mm/restore.macro), another XML file. 
This file could be used to restore the device to the state it was when the file was recorded. 

---
## Tasks:

1. Double-click on the table above to edit it.
2. Open [restore.macro](../data/theiere-Fe-10mm/restore.macro).
3. Some information is redundant and it is worth double-checking if possible.
    - Identify the object named `ExtendedTomo`.
        - Is the imager the same as what we expected (i.e. `Imager_2`)?
        - Is the number of images per turn the same too (i.e. 1504)?
    - Identify the object named `XRay300_2`.
        - What about the voltage,
        - and the current?
        - Was the beam on (see `xRayEnabled`)?
4. Have a look at the photograph above. Can you notice that something is held between the source and the sample? It is the filter that will alter the beam spectrum in order to reduce the relative amount of low energy photons, i.e. reduce beamhardening.
5. Go back to [restore.macro](../data/theiere-Fe-10mm/restore.macro) and checkout all the fields of the object names `XRay300_2`.
    - Identify the filter.
    - In the cell below, uncomment all the lines of code, and input the values of `filter_material`, `filter_thickness`, and `thickness_unit`. Note that `filter_material` and `thickness_unit` are strings; `filter_thickness` int or float.
    - Report the corresponding information in the table.
---

In [ ]:
# filter_material = ""
# filter_thickness = 
# thickness_unit = ""

# filtration = [
#     [filter_material, filter_thickness, thickness_unit]
# ]

In [ ]:
# %load 'snippets/single_material-reproducing_CT_scan-JSON_file-03.py'

There is one more thing we need: the exposure time. We'll be able to generate the right amount of noise with this information.
The exposure is related to the detector/imager. We will record the exposure in seconds and compute it in terms of mAs (the useful one to control the noise).

---
## Tasks:

- Identify the object names `Imager_2` in [restore.macro](../data/theiere-Fe-10mm/restore.macro).
- What is the exposure?
- It is an index in a lookup table, which is provided below.
- Uncomment the line with `exposure_ID = ` and input the corresponding index.
- Compute the exposure in milliampereseconds, i.e. the product of the current in mA and exposure time in seconds.
- Report the corresponding information in the table.
---

In [ ]:
exposure_lookup_table = [0.067, 0.083, 0.111, 0.167, 0.333, 0.500] # [s]
# exposure_ID = 
exposure_time_in_sec = exposure_lookup_table[exposure_ID - 1]
mAs = exposure_time_in_sec * current_in_uA / 1000

print("Exposure time:", exposure_time_in_sec, "in sec")
print("Exposure:", mAs, "mAs")

In [ ]:
# %load 'snippets/single_material-reproducing_CT_scan-JSON_file-04.py'

We have now collected all the information we needed from the metadata. We are still missing two pieces of information. The image size and the pixel pitch (space between the centre of two consecutive pixels). I can help you, it's 150 &mu;m along both directions.

---
## Task:

- Report the corresponding information in the table.
---

In [ ]:
pixel_pitch_in_um = np.array([150, 150])

We also compute the magnification as it will give a sence of the sample size when displaying radiographs.

In [ ]:
magnification = sdd_in_mm / sod_in_mm;
print("Magnification:", magnification)
spacing_in_mm = (pixel_pitch_in_um / magnification) / 1000.0;

For the image size, we'll open and display a radiograph. 

In [ ]:
raw_radiograph_reference = np.array(imread(os.path.join(dataset_path, projection_fname)), dtype=np.single)

# Average of the pixel intensity over the first 400 rows, i.e. no object
white_value = raw_radiograph_reference[:400,:].mean()

# Normalisation
raw_radiograph_reference /= white_value

---
## Tasks:

- Run the cell below,
- Report the corresponding information in the table.
---

In [ ]:
projection_number_of_rows = raw_radiograph_reference.shape[0]
projection_number_of_cols = raw_radiograph_reference.shape[1]

print("Image size:", str(projection_number_of_cols) + "x" + str(projection_number_of_rows), "pixels")

In [ ]:
# %load 'snippets/single_material-reproducing_CT_scan-JSON_file-05.py'

and display it either as an attenuation image or absorption image (using the $-\log$ of the attenuation).

In [ ]:
# Plot the two images side-by-side
fig = plt.figure(figsize=(15, 7))
gs = gridspec.GridSpec(1, 2)
axes = [fig.add_subplot(ss) for ss in gs]

extent=[0,(raw_radiograph_reference.shape[1]-1)*spacing_in_mm[0],0,(raw_radiograph_reference.shape[0]-1)*spacing_in_mm[1]]

im1 = axes[0].imshow(raw_radiograph_reference, cmap="gray", extent=extent)
im2 = axes[1].imshow(-np.log(raw_radiograph_reference), cmap="gray", extent=extent)

plt.suptitle("Digital radiograph of a teapot")

axes[0].set_title("Attenuation")
axes[1].set_title("Absorption")

axes[0].set_xlabel("Pixel position\n(in mm)")
axes[1].set_xlabel("Pixel position\n(in mm)")
axes[0].set_ylabel("Pixel position\n(in mm)")

plt.tight_layout()

plt.savefig(output_path + '/reference-radiograph.pdf')
plt.savefig(output_path + '/reference-radiograph.png')

# 2. Describe the simulation and reconstruction in a human-friendly file

We will populate the file [../data/theiere-Fe-10mm/parameters.json](../data/theiere-Fe-10mm/parameters.json) with information from our table. 
Note, when file or directory paths are given, they are relative to the path of the JSON file. 

- The file typically starts with:

    ```json
    {
        "File format version": [1, 0, 0],
        "Window size": [500, 500],

        ...
    }
    ```
    It is to initialise the simulation engine and the 3D visualisation environment.

- It is followed by the detector. The JSON format would be:
    ```json
    {
        ...

        "Detector": {
            "Position": [0.0, 0.0, MINUS ODD, "mm"],
            "UpVector": [0.0, -1.0, 0.0],
            "RightVector": [-1.0, 0.0, 0.0],
            "NumberOfPixels": [COLS, ROWS],
            "Spacing": [PITCH, PITCH, "um"],
            "Scintillator": {
                "Material": "Gd2O2S DRZ-Plus",
                "Thickness": 210,
                "Unit": "um"
            }
        },
        
        ...
    }
    ```

---
## Tasks:

- To speeds thing up, particularly on laptops or on Cloud instances without OpenGL support (e.g. Google Colab), 
    - multiply the pixel pitch by 4 and 
    - divide the number of pixels by 4. Make sure to use an integer division!
- Open and edit [../data/theiere-Fe-10mm/parameters.json](../data/theiere-Fe-10mm/parameters.json) to
- Replace with their respective values:
    - `MINUS ODD`, 
    - `COLS`, 
    - `ROWS`, and
    - `PITCH`.
---

- We can now worry about the source. The JSON format would be:

    ```json
    {
        ...
        
        "Source": {
            "Position": [0.0, 0, SOD, "mm"],
            "Shape": "CONEBEAM",
            "Beam": {
                "Peak kilo voltage": VOLTAGE_IN_kV,
                "Tube angle": 12.0,
                "mAs": mAS_IF_KNOWN,
                "filter": [
                    ["SYMBOL_OF_FILTER_MATERIAL", FILTER_THICKNESS, "THICKNESS_UNIT"]
                ],
                "MaxNumberOfEnergyBins": 50
            }
        },

        ...

    }
    ```

    Note that the shape could be `PARALLEL` to mimick synchrotron radiation. There are also other ways to specify the beam spectrum. `maxNumberOfEnergyBins` is used to reduce the number of energy bins in the spectrum without altering the flux. If it is too high, and the resoultion too high, then the simulation will be very slooOoooOoow.

---
## Tasks:

Replace with their respective values:

- `SOD`, 
- `VOLTAGE_IN_kV`, 
- `mAS_IF_KNOWN`,
- `SYMBOL_OF_FILTER_MATERIAL`,
- `FILTER_THICKNESS`, and 
- `THICKNESS_UNIT`.
---

- We can now add our beautiful teapot. It is made of cast iron. Its density is about 6.675 g.cm<sup>-3</sup>. The material composition is roughly:

    - Iron (Z=26): 95% of the alloy,
    - Carbon (Z=6): 3%,
    - Silicon: 2%.

    As the teapot is cast iron, it's matt black and we can set the colour for the 3D visualisation. To make things easy, we move the samples to the centre of the world. We also rotate the teapot so that it is oriented as it was in the experimental scan.

```json
{
    ...

        "Samples": [
            "MoveToCentre",
            
            {
                "Label": "Teapot",
                "Path": "models-teapot-decimated.stl",
                "Unit": "mm",
                "Material": [
                    "Mixture", [
                        26, 0.95,
                        6, 0.03,
                        14, 0.02
                    ]
                ],
                "Density": 5.5,
                "AmbientColour": [ 0.20, 0.20, 0.20, 1.0 ],
                "DiffuseColour": [ 0.0, 0.0, 0.0, 1.0 ],
                "SpecularColour": [ 0.0, 0.0, 0.0, 1.0 ],
                "Shininess": 20.0,
                "Transform": [
                    ["Rotation", -90, 1, 0, 0]
                ]
            }
        ],


    ...
}
```

- We finally describe the CT acquisition with:

    ```json
    {
        ...
        
        "Scan": {
            "NumberOfProjections": NUMBER_OF_PROJECTION,
            "FinalAngle": 360,
            "IncludeFinalAngle": false,
            "CenterOfRotation": [0,0,0],
            "RotationAxis": [SAME_AS_UpVector_OF_DETECTOR],
            "OutFolder": "../notebooks/output_data/single_material-reproducing_CT_scan-JSON_file/Proj-simulated"
        }
    }

---
## Tasks:

Replace with their respective values:

- `NUMBER_OF_PROJECTION`, and 
- `SAME_AS_UpVector_OF_DETECTOR`.
---

# 3. Run the simulation using this file

Initialise the simulation engine and the 3D visualisation environment.

In [ ]:
#JSON_fname = os.path.join(dataset_path, "parameters-solution.json")
JSON_fname = os.path.join(dataset_path, "parameters.json")
json2gvxr.initGVXR(JSON_fname)

We can now load the detector properties.

In [ ]:
json2gvxr.initDetector(verbose=1)

Display the energy response of the detector corresponding to the scintillator.

In [ ]:
detector_response = np.array(gvxr.getEnergyResponse("keV"));

plt.figure(figsize= (20,10))
# plt.title("Detector response")
plt.plot(detector_response[:,0], detector_response[:,1])
plt.xlabel('Incident energy: E (in keV)')
plt.ylabel('Detector energy response: $\\delta$(E) (in keV)')

plt.tight_layout()

plt.savefig(output_path + '/detector_response.pdf')
plt.savefig(output_path + '/detector_response.png')

We can now load the geometrical source properties.

In [ ]:
json2gvxr.initSourceGeometry(verbose=1)

and the spectrum.

In [ ]:
spectrum, unit, k, f = json2gvxr.initSpectrum()

In [ ]:
plt.figure(figsize= (20,10))
# plt.title("Beam spectrum")
plt.bar(k, f, width=1)
plt.xlabel('Energy in keV')
plt.ylabel('Probability distribution of photons per keV')
plt.savefig(output_path + "/spectrum.pdf")
plt.show()

If you have enough computational power, you may enable Poisson noise. It'll be based on the exposure in mAs. If you use the same pixel resolution as the experimental data, you'll see it is quite realistic!

In [ ]:
gvxr.disablePoissonNoise()
# gvxr.enablePoissonNoise()

We load the sample.

In [ ]:
json2gvxr.initSamples(verbose=1)

We are now ready to compute an X-ray image and display it.

In [ ]:
x_ray_image = np.array(gvxr.computeXRayImage(), dtype=np.single) / gvxr.getTotalEnergyWithDetectorResponse();
gvxr.displayScene()

We can display it side-by-side with the experimental image.

In [ ]:
# Plot the two images side-by-side
fig = plt.figure(figsize=(15, 7))
gs = gridspec.GridSpec(1, 2)
axes = [fig.add_subplot(ss) for ss in gs]

extent=[0,(raw_radiograph_reference.shape[1]-1)*spacing_in_mm[0],0,(raw_radiograph_reference.shape[0]-1)*spacing_in_mm[1]]

im1 = axes[0].imshow(raw_radiograph_reference, cmap="gray", extent=extent, vmin=0, vmax=1)
im2 = axes[1].imshow(x_ray_image, cmap="gray", extent=extent, vmin=0, vmax=1)

axes[0].plot([extent[1] // 2, extent[1] // 2], [0, extent[3]])
axes[1].plot([extent[1] // 2, extent[1] // 2], [0, extent[3]])

plt.suptitle("Digital radiograph of a teapot")

axes[0].set_title("Experimental")
axes[1].set_title("Simulated")

axes[0].set_xlabel("Pixel position\n(in mm)")
axes[1].set_xlabel("Pixel position\n(in mm)")
axes[0].set_ylabel("Pixel position\n(in mm)")

axes[0].set_ylim(19, extent[3])
axes[1].set_ylim(19, extent[3])

plt.tight_layout()

plt.savefig(output_path + '/difference-radiograph.pdf')
plt.savefig(output_path + '/difference-radiograph.png')

and compare intensity profiles.

In [ ]:
# Plot the two images side-by-side
fig = plt.figure(figsize=(10, 7))

plt.plot(np.linspace(0,(raw_radiograph_reference.shape[1]-1)*spacing_in_mm[0], raw_radiograph_reference.shape[1]),
         raw_radiograph_reference[:, raw_radiograph_reference.shape[1] // 2], 
         label="Experimental")

plt.plot(np.linspace(0,(raw_radiograph_reference.shape[1]-1)*spacing_in_mm[0], x_ray_image.shape[1]),
         x_ray_image[:, x_ray_image.shape[1] // 2],
         label="Simulated")

plt.title("Intensity profiles")
plt.legend()
plt.tight_layout()

plt.savefig(output_path + '/difference-radiograph-profiles.pdf')
plt.savefig(output_path + '/difference-radiograph-profiles.png')

The main difference may be explained:
- The material composition is actually unknown. We used average values found on the web.
- The simulation is not registered on the experimental data.
- The impulse response of the detector has been ignored (it could be added if known).
- The focal spot of the tube has been ignored (it could be added if known).
- If Poisson noise was enable, it would have been reduced compared to the experimental data as we downsampled the detector. Same number of photons per cm<sup>2</sup> but a lot more per pixels.

It may be useful to visualise the 3D environment to ascertain everything is as expected. We use k3D if possible. It's a nice 3D visualisation framework for Jupyter notebooks.

In [ ]:
if IN_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

plot = visualise(use_log=True, use_negative=True, sharpen_ksize=2, sharpen_alpha=1.0);

if plot:
    plot.display();

In [ ]:
if plot:
    plot.fetch_screenshot();

In [ ]:
if plot:
    screenshot_fname = os.path.join(output_path, "k3d_screenshot.png");
    if not os.path.exists(screenshot_fname):
        k3d_screenshot = plot.screenshot;
        data = base64.b64decode(k3d_screenshot);
        with open(screenshot_fname, "wb") as fp:
            fp.write(data);
            fp.flush();
            fp.close();

if IN_COLAB:
    output.disable_custom_widget_manager()

Compute the X-ray projections for a CT acquisition.

In [ ]:
json2gvxr.initScan()
angles=json2gvxr.doCTScan()


# 4. Perform the reconstruction using CIL

In [ ]:
# Read the simulated data with CIL.
reader = JSON2gVXRDataReader(JSON_fname);
data = reader.read()

In [ ]:
print("data.geometry", data.geometry)

In [ ]:
show2D(data, origin='upper-left')

In [ ]:
islicer(data, origin='upper-left')

In [ ]:
data_corr = TransmissionAbsorptionConverter(white_level=data.max())(data)

In [ ]:
show2D(data_corr, origin='upper-left')

In [ ]:
islicer(data_corr, origin='upper-left')

In [ ]:
data_corr.reorder(order='tigre')

In [ ]:
ig = data_corr.geometry.get_ImageGeometry();

print("Image geometry", ig)

In [ ]:
# Perform the reconstruction with CIL
FDK_reconstruction = FDK(data_corr, ig).run()

In [ ]:
show2D(FDK_reconstruction, slice_list=[1, FDK_reconstruction.shape[0] // 2, FDK_reconstruction.shape[0] - 2], title=["2nd slice", "Middle slice", "Slice before last"], num_cols=3)

Save the reconstructed slices as a TIFF stack using 16-bit unsigned integers.

In [ ]:
# Save the reconstructed CT images
writer = TIFFWriter(data=FDK_reconstruction, file_name=os.path.join(output_path, "recons-" + str(nb_images), "slice_"), compression="uint16");
writer.write();

Explore the reconstructed volume and illustrate the partial view artefacts due to the partial view effect on the first and last slices.

In [ ]:
islicer(FDK_reconstruction, minmax=None)

# Further work

The 10 mm of iron has dramatically attenuated the beam hardening. We could try to generate a lot of artefacts.

---
## Task:

- Delete the filter from the JSON file, and
- Re-run the notebook.

# Cleaning up

Once we have finished it is good practice to clean up the OpenGL contexts and windows with the following command.

In [ ]:
gvxr.terminate();